In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/cmi_inference_server.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/cmi_gateway.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/__init__.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/templates.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/base_gateway.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/relay.py
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/cmi-detect-behavior-with-sensor-data/kaggle_evaluation/core/__init__.py
/kaggle/input/cmi-detect-behav

In [2]:
import numpy as np
import pandas as pd


In [3]:
import pandas as pd

train = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv", low_memory=False)
test = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv", low_memory=False)
train_demo = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv")
test_demo = pd.read_csv("/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv")

print(train.shape, test.shape)
train.head()

(574945, 341) (107, 336)


,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,-1.0,-1.0,112.0,119.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,-1.0,-1.0,101.0,111.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,-1.0,-1.0,101.0,109.0,125.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [4]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.fft import fft
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import os

# === CONFIGURATION ===
# Using the user-provided file paths
TRAIN_CSV = '/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv'
TEST_CSV = '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv'
DEMOG_TRAIN = '/kaggle/input/cmi-detect-behavior-with-sensor-data/train_demographics.csv'
DEMOG_TEST = '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv'

FEATURE_OUT = '/kaggle/working/sequence_features.csv'
RANDOM_SEED = 42
WINDOW_SIZE = 50 # A tunable hyperparameter

# Define sensor column groups
ACC_COLS = ['acc_x', 'acc_y', 'acc_z']
ROT_COLS = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
IMU_COLS = ACC_COLS + ROT_COLS
THM_COLS = [f'thm_{i}' for i in range(1, 6)]
TOF_COLS = [f'tof_{i}_v{j}' for i in range(1, 6) for j in range(0, 64)]


print("DONE")


DONE


In [5]:
# STEP 1: FEATURE ENGINEERING FUNCTIONS
# ===============================================

def extract_imu_features(window_df):
    """Extracts Time-Domain, Frequency-Domain, and statistical IMU features."""
    features = {}
    
    # ... (other code for Time-Domain features)
    
    # Time-Domain and Statistical Features (CORRECTED SECTION)
    for col in IMU_COLS:
        col_prefix = col
        features[f'{col_prefix}_mean'] = window_df[col].mean()
        features[f'{col_prefix}_var'] = window_df[col].var()
        
        # Explicitly check sample size before calculating skew and kurtosis
        data = window_df[col].dropna()
        MIN_SAMPLE_SIZE = 8 # Set minimum required samples (e.g., 8 is a common threshold)
        
        if len(data) >= MIN_SAMPLE_SIZE:
            features[f'{col_prefix}_skew'] = skew(data)
            features[f'{col_prefix}_kurt'] = kurtosis(data)
        else:
            # If sample is too small, set to NaN (which the imputer will handle later)
            features[f'{col_prefix}_skew'] = np.nan
            features[f'{col_prefix}_kurt'] = np.nan
    #Signal Magnitude Area (SMA) and Total Magnitude
    acc_mag = np.sqrt(np.square(window_df[ACC_COLS]).sum(axis=1))
    features['acc_SMA'] = acc_mag.sum() / WINDOW_SIZE
    features['acc_mag_mean'] = acc_mag.mean()
    features['acc_mag_var'] = acc_mag.var()

    # Cross-Axis Correlation
    features['acc_corr_xy'] = window_df['acc_x'].corr(window_df['acc_y'])
    
    # Frequency-Domain Features (FFT magnitude of first few non-DC components)
    for col in IMU_COLS:
        if len(window_df[col].dropna()) > 0:
            fft_values = np.abs(fft(window_df[col].fillna(window_df[col].mean()).values))
            for i in range(1, 6):
                features[f'{col}_fft_mag_{i}'] = fft_values[i]
        else:
             for i in range(1, 6):
                features[f'{col}_fft_mag_{i}'] = np.nan
            
    return features

In [6]:
def extract_thermopile_features(window_df):
    """Extracts features from Thermopile data (thm_1 to thm_5), including gradients."""
    features = {}

    # Simple Statistical Features (Average Heat Intensity, Temporal Variance)
    for col in THM_COLS:
        col_prefix = col
        features[f'{col_prefix}_mean'] = window_df[col].mean()
        features[f'{col_prefix}_var'] = window_df[col].var()

    # Thermopile Gradient Features (Difference between adjacent sensors)
    for i in range(1, 5):
        grad_col = f'thm_grad_{i}_{i+1}'
        gradient = window_df[f'thm_{i}'] - window_df[f'thm_{i+1}']
        features[f'{grad_col}_mean'] = gradient.mean()
        features[f'{grad_col}_var'] = gradient.var()
        
    return features

In [7]:
def extract_tof_features(window_df):
    """Extracts spatial and temporal features from Time-of-Flight (ToF) data."""
    features = {}

    # Replace the 'no sensor response' value (-1) with NaN for statistics
    tof_data = window_df[TOF_COLS].replace(-1, np.nan)
    
    # Overall Proximity Features
    features['tof_min_proximity'] = tof_data.min().min() # Closest object
    features['tof_max_proximity'] = tof_data.max().max() # Farthest object
    
    # Per-Sensor (5 sensors) Spatial and Temporal Features
    for i in range(1, 6):
        sensor_cols = [f'tof_{i}_v{j}' for j in range(0, 64)]
        tof_sensor_data = tof_data[sensor_cols]
        prefix = f'tof_{i}'

        # 1. Spatial Statistics
        spatial_mean_series = tof_sensor_data.mean(axis=1) # Mean distance per row
        features[f'{prefix}_spatial_mean'] = spatial_mean_series.mean()
        features[f'{prefix}_spatial_var'] = spatial_mean_series.var()
        
        # 2. Number of 'Active' Pixels (Non-negative)
        active_pixels = (window_df[sensor_cols] != -1).sum(axis=1)
        features[f'{prefix}_active_pixels_mean'] = active_pixels.mean()
        features[f'{prefix}_active_pixels_var'] = active_pixels.var()

        # 3. Temporal Slope / Trend
        avg_proximity_series = spatial_mean_series.dropna()
        if len(avg_proximity_series) >= 2:
            # Calculate temporal slope (change in average distance over time)
            slope = np.polyfit(np.arange(len(avg_proximity_series)), avg_proximity_series, 1)[0]
            features[f'{prefix}_temporal_slope'] = slope
        else:
            features[f'{prefix}_temporal_slope'] = np.nan

    return features

In [8]:
# STEP 2: DATA PIPELINE (Windowing, Extraction, Merging)
# ===============================================

def run_feature_pipeline(ts_path, demog_path, is_train=True):
    """Loads, processes time-series data, and merges with demographics."""
    print(f"--- Running Pipeline for {'Train' if is_train else 'Test'} Data ---")
    
    # Load Time-Series and Demographics
    df_ts = pd.read_csv(ts_path)
    df_demog = pd.read_csv(demog_path)
    
    features_list = []
    
    # Iterate through each unique sequence
    sequence_groups = df_ts.groupby('sequence_id')
    for seq_id, group in sequence_groups:
        
        # --- Windowing Strategy ---
        # For a full sequence (Transition, Pause, Gesture), the critical event 
        # is the Gesture, which is at the end of the sequence. We use the last 
        # WINDOW_SIZE rows to capture this final motion.
        window_df = group.tail(WINDOW_SIZE).copy()
        
        if len(window_df) < WINDOW_SIZE:
            # Skip short sequences that can't form a reliable feature vector
            continue
            
        # 1. Feature Extraction
        features = {}
        features.update(extract_imu_features(window_df))
        features.update(extract_thermopile_features(window_df))
        features.update(extract_tof_features(window_df))
        
        # 2. Add identifying and target columns
        features['sequence_id'] = seq_id
        features['subject'] = window_df['subject'].iloc[0]
        if is_train:
            # The gesture column is only present in the training set
            features['gesture'] = window_df['gesture'].iloc[-1] 
        
        features_list.append(features)
        
    df_features = pd.DataFrame(features_list)
    
    # 3. Merge with Demographics (Static Features Fusion)
    df_final = pd.merge(df_features, df_demog, on='subject', how='left')
    
    print(f"Generated {len(df_final)} feature vectors. Total features: {len(df_final.columns)}")
    return df_final

In [9]:
# Execute pipeline for both datasets
df_train_features = run_feature_pipeline(TRAIN_CSV, DEMOG_TRAIN, is_train=True)
df_test_features = run_feature_pipeline(TEST_CSV, DEMOG_TEST, is_train=False)

# Save the features (optional, for debugging/reuse)
df_train_features.to_csv(FEATURE_OUT.replace(".csv", "_train.csv"), index=False)

--- Running Pipeline for Train Data ---
Generated 6819 feature vectors. Total features: 122
--- Running Pipeline for Test Data ---
Generated 2 feature vectors. Total features: 121


In [10]:
# STEP 3: MODELING PIPELINE (Imputation, Scaling, Training)
# ===============================================

def train_and_evaluate_model(df_train_features, df_test_features, random_seed):
    """Prepares data, trains Random Forest model, and prepares submission."""
    
    # --- Data Preparation ---
    # 1. Separate Features (X) and Target (y)
    X_train = df_train_features.drop(columns=['sequence_id', 'subject', 'gesture'])
    y_train = df_train_features['gesture']
    
    # Align test features to match training feature columns
    X_test_seq_id = df_test_features['sequence_id']
    X_test = df_test_features.drop(columns=['sequence_id', 'subject'])
    
    # 2. Define the Model Pipeline
    IMPUTATION_STEP = ('imputer', SimpleImputer(strategy='median')) # Handles NaNs from feature engineering
    
    # Random Forest (Tree-based model, handles complexity, with class weighting)
    rf_pipe = Pipeline([
        IMPUTATION_STEP, 
        ('scaler', StandardScaler()), # Scaling helps with consistent feature contribution
        ('clf', RandomForestClassifier(
            n_estimators=500, # Increased n_estimators for better performance
            random_state=random_seed,
            class_weight='balanced', # Crucial for handling imbalanced BFRB/Non-BFRB classes
            n_jobs=-1 # Use all cores
        ))
    ])
    
    # --- Training and Prediction ---
    print("\nStarting Model Training...")
    rf_pipe.fit(X_train, y_train)
    print("Training Complete.")
    
    # Predict on Test Data
    y_pred_test = rf_pipe.predict(X_test)
    
    # --- Evaluation (Conceptual: Needs cross-validation for real score) ---
    # For a quick check, predict on training data:
    y_pred_train = rf_pipe.predict(X_train)
    macro_f1_train = f1_score(y_train, y_pred_train, average='macro')
    print(f"Training Macro F1 Score (Validation Check): {macro_f1_train:.4f}")
    
    # --- Submission File Generation ---
    df_submission = pd.DataFrame({
        'sequence_id': X_test_seq_id,
        'gesture': y_pred_test
    })
    
    # Final Output file path for Kaggle submission
    submission_path = os.path.join('/kaggle/working', 'submission.csv')
    df_submission.to_csv(submission_path, index=False)
    print(f"\nSubmission file created at: {submission_path}")
    print(df_submission.head())

# Execute the final modeling step
train_and_evaluate_model(df_train_features, df_test_features, RANDOM_SEED)


Starting Model Training...
Training Complete.
Training Macro F1 Score (Validation Check): 1.0000

Submission file created at: /kaggle/working/submission.csv
  sequence_id              gesture
0  SEQ_000001   Forehead - scratch
1  SEQ_000011  Eyelash - pull hair


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier # NEW
from sklearn.neighbors import KNeighborsClassifier # NEW
from sklearn.ensemble import RandomForestClassifier, VotingClassifier # UPDATED
from sklearn.metrics import f1_score, confusion_matrix
import os

# --- Imbalance Handling Note ---
# Class Weighting (class_weight='balanced') is used for LR, DT, and RF as it's built-in 
# and often preferred inside pipelines. SMOTE, though effective, requires an external 
# library (imblearn) and must be applied BEFORE scaling, which complicates the pipeline.

def train_and_evaluate_model(df_train_features, df_test_features, random_seed):
    """Defines, trains, and evaluates all baseline, tree-based, and ensemble models."""
    
    print("\nStarting Model Development and Evaluation...")
    
    # --- Data Preparation ---
    X_train_full = df_train_features.drop(columns=['sequence_id', 'subject', 'gesture'])
    y_train_full = df_train_features['gesture']
    
    # Use a split of the TRAIN data for internal evaluation (Macro F1)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_full, y_train_full, test_size=0.3, random_state=random_seed, stratify=y_train_full
    )
    
    # Prepare Test Data for final predictions
    X_test_seq_id = df_test_features['sequence_id']
    X_test = df_test_features.drop(columns=['sequence_id', 'subject'])
    
    # --- Pipeline Components ---
    IMPUTATION_STEP = ('imputer', SimpleImputer(strategy='median'))
    SCALING_STEP = ('scaler', StandardScaler())
    
    
    # --- 1. Baseline Models ---
    
    # 1A. Logistic Regression (Imbalanced handled by class_weight)
    lr_pipe = Pipeline([
        IMPUTATION_STEP, SCALING_STEP,
        ('clf', LogisticRegression(random_state=random_seed, solver='liblinear', multi_class='ovr', class_weight='balanced'))
    ])
    
    # 1B. k-Nearest Neighbors (No inherent imbalance handling, scaling is crucial)
    knn_pipe = Pipeline([
        IMPUTATION_STEP, SCALING_STEP,
        ('clf', KNeighborsClassifier(n_neighbors=5)) # n_neighbors is tunable
    ])
    
    
    # --- 2. Tree-based Models ---
    
    # 2A. Decision Tree (Imbalanced handled by class_weight)
    dt_pipe = Pipeline([
        IMPUTATION_STEP,
        ('clf', DecisionTreeClassifier(random_state=random_seed, class_weight='balanced'))
    ])
    
    # 2B. Random Forest (Imbalanced handled by class_weight)
    rf_pipe = Pipeline([
        IMPUTATION_STEP,
        ('clf', RandomForestClassifier(n_estimators=500, random_state=random_seed, class_weight='balanced', n_jobs=-1))
    ])
    
    
    # --- 3. Ensemble Model: Voting Classifier ---
    
    # Use the best performing and most diverse base models: LR, RF, and DT.
    # The 'named_estimators' must use the name of the step defined above.
    
    
    # Wrap the ensemble in a pipeline for consistent data preprocessing
    voting_pipe = VotingClassifier(
        estimators=[
            ('lr', lr_pipe),
            ('rf', rf_pipe),
            ('dt', dt_pipe)
        ],
        voting='soft', 
        n_jobs=-1
    )
    
    
    # --- 4. Training and Evaluation Loop ---
    
    models = {
        "Logistic Regression": lr_pipe,
        "k-Nearest Neighbors": knn_pipe,
        "Decision Tree": dt_pipe,
        "Random Forest": rf_pipe,
        "Voting Classifier": voting_pipe
    }
    
    best_model_name = ""
    best_f1 = -1.0
    
    results = {}
    for name, pipe in models.items():
        print(f"\nTraining {name}...")
        pipe.fit(X_train, y_train)
        y_pred_val = pipe.predict(X_val)
        
        macro_f1 = f1_score(y_val, y_pred_val, average='macro')
        results[name] = macro_f1
        print(f"{name} Macro F1 Score (Validation): {macro_f1:.4f}")

        if macro_f1 > best_f1:
            best_f1 = macro_f1
            best_model_name = name
            
    # --- 5. Final Prediction using the BEST model ---
    
    final_model = models[best_model_name]
    
    # Retrain the best model on the FULL training set for best performance
    print(f"\nRetraining BEST model ({best_model_name}) on full training data...")
    final_model.fit(X_train_full, y_train_full)
    
    y_pred_test = final_model.predict(X_test)
    
    # --- Submission File Generation ---
    df_submission = pd.DataFrame({
        'sequence_id': X_test_seq_id,
        'gesture': y_pred_test
    })
    
    submission_path = os.path.join('/kaggle/working', 'submission_ensemble.csv')
    df_submission.to_csv(submission_path, index=False)
    
    print("\n--- Summary of Validation F1 Scores ---")
    for name, f1 in results.items():
         print(f"{name}: {f1:.4f}")
         
    print(f"\nFinal prediction made using: {best_model_name}")
    print(f"Submission file created at: {submission_path}")
    print(df_submission.head())
    
# Execute the final modeling step
train_and_evaluate_model(df_train_features, df_test_features, RANDOM_SEED)


Starting Model Development and Evaluation...

Training Logistic Regression...
Logistic Regression Macro F1 Score (Validation): 0.5029

Training k-Nearest Neighbors...
k-Nearest Neighbors Macro F1 Score (Validation): 0.4049

Training Decision Tree...
Decision Tree Macro F1 Score (Validation): 0.4203

Training Random Forest...
Random Forest Macro F1 Score (Validation): 0.5629

Training Voting Classifier...
Voting Classifier Macro F1 Score (Validation): 0.4526

Retraining BEST model (Random Forest) on full training data...

--- Summary of Validation F1 Scores ---
Logistic Regression: 0.5029
k-Nearest Neighbors: 0.4049
Decision Tree: 0.4203
Random Forest: 0.5629
Voting Classifier: 0.4526

Final prediction made using: Random Forest
Submission file created at: /kaggle/working/submission_ensemble.csv
  sequence_id              gesture
0  SEQ_000001   Forehead - scratch
1  SEQ_000011  Eyelash - pull hair


In [12]:
# Create a dtype map to save memory:
# Most sensor data can be safely cast to float32.
DTYPE_MAP = {
    'row_id': 'object',
    'sequence_id': 'object',
    'sequence_counter': 'int16',
    'subject': 'object',
    # IMU & THM: Use float32
    **{col: 'float32' for col in ACC_COLS + ROT_COLS + THM_COLS},
    # ToF: They are uncalibrated sensor values (0-254) but loaded as float/int. 
    # Use float32 to be safe with operations.
    **{col: 'float32' for col in TOF_COLS}, 
    # Target columns (for train only)
    'sequence_type': 'object', 
    'gesture': 'object', 
    'orientation': 'object',
    'behavior': 'object'
}


def run_feature_pipeline_optimized(ts_path, demog_path, is_train=True):
    """Loads data with optimized dtypes, processes, and merges with demographics."""
    print(f"--- Running Optimized Pipeline for {'Train' if is_train else 'Test'} Data ---")
    
    # Load Time-Series with optimized dtypes
    # We load only the columns we need to save even more memory
    cols_to_load = ['sequence_id', 'subject', 'sequence_counter'] + IMU_COLS + THM_COLS + TOF_COLS
    if is_train:
         cols_to_load += ['gesture']
    
    # Filter DTYPE_MAP to only include columns being loaded
    dtype_filter = {k: v for k, v in DTYPE_MAP.items() if k in cols_to_load}
    
    # Load the time-series file with explicit dtypes
    df_ts = pd.read_csv(ts_path, dtype=dtype_filter, usecols=cols_to_load)
    df_demog = pd.read_csv(demog_path)
    
    # ... (rest of the run_feature_pipeline function remains the same)
    
    features_list = []
    
    # Iterate through each unique sequence
    sequence_groups = df_ts.groupby('sequence_id')
    for seq_id, group in sequence_groups:
        # ... (feature extraction code remains the same)
        window_df = group.tail(WINDOW_SIZE).copy()
        
        if len(window_df) < WINDOW_SIZE:
            continue
            
        features = {}
        features.update(extract_imu_features(window_df))
        features.update(extract_thermopile_features(window_df))
        features.update(extract_tof_features(window_df))
        
        features['sequence_id'] = seq_id
        features['subject'] = window_df['subject'].iloc[0]
        if is_train:
            features['gesture'] = window_df['gesture'].iloc[-1] 
        
        features_list.append(features)
        
    df_features = pd.DataFrame(features_list)
    
    # Crucial Memory Step: Explicitly delete large dataframe after features are extracted
    del df_ts
    
    # Merge with Demographics
    df_final = pd.merge(df_features, df_demog, on='subject', how='left')
    
    print(f"Generated {len(df_final)} feature vectors. Total features: {len(df_final.columns)}")
    return df_final

In [13]:
# Execute pipeline for both datasets using the optimized function
df_train_features = run_feature_pipeline_optimized(TRAIN_CSV, DEMOG_TRAIN, is_train=True)
df_test_features = run_feature_pipeline_optimized(TEST_CSV, DEMOG_TEST, is_train=False)

# Save the features
# Saving to disk ensures the memory is cleared and the feature set is usable even if later steps fail.
df_train_features.to_csv(FEATURE_OUT.replace(".csv", "_train.csv"), index=False)
df_test_features.to_csv(FEATURE_OUT.replace(".csv", "_test.csv"), index=False)

# Execute the final modeling step (assuming train_and_evaluate_model is defined correctly)
train_and_evaluate_model(df_train_features, df_test_features, RANDOM_SEED)

--- Running Optimized Pipeline for Train Data ---
Generated 6819 feature vectors. Total features: 122
--- Running Optimized Pipeline for Test Data ---
Generated 2 feature vectors. Total features: 121

Starting Model Development and Evaluation...

Training Logistic Regression...
Logistic Regression Macro F1 Score (Validation): 0.5029

Training k-Nearest Neighbors...
k-Nearest Neighbors Macro F1 Score (Validation): 0.4049

Training Decision Tree...
Decision Tree Macro F1 Score (Validation): 0.4203

Training Random Forest...
Random Forest Macro F1 Score (Validation): 0.5632

Training Voting Classifier...
Voting Classifier Macro F1 Score (Validation): 0.4526

Retraining BEST model (Random Forest) on full training data...

--- Summary of Validation F1 Scores ---
Logistic Regression: 0.5029
k-Nearest Neighbors: 0.4049
Decision Tree: 0.4203
Random Forest: 0.5632
Voting Classifier: 0.4526

Final prediction made using: Random Forest
Submission file created at: /kaggle/working/submission_ensembl

In [14]:
h=pd.read_csv('/kaggle/working/submission.csv')
print(h)

  sequence_id              gesture
0  SEQ_000001   Forehead - scratch
1  SEQ_000011  Eyelash - pull hair
